In [1]:
import pandas as pd
import numpy as np

In [13]:
PADDINGS = 10

In [3]:
splits = {'train': 'train.csv', 'validation': 'dev.csv', 'test': 'test.csv'}
df = pd.read_csv("hf://datasets/ibm-research/argument_quality_ranking_30k/" + splits["train"])

In [7]:
argument = list(df.argument)
topic = list(df.topic)
note = list(df.WA)

In [5]:
## train test split
from sklearn.model_selection import train_test_split

In [17]:
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec

nltk.download('punkt')

data = []
for i in range(len(argument)):
    arg_token = word_tokenize(argument[i].lower())
    arg_topic = word_tokenize(topic[i].lower())
    if len(arg_topic) < PADDINGS:
        arg_topic += ['<pad>'] * (PADDINGS - len(arg_topic))

    data.append(arg_topic + arg_token)

model_embedding = Word2Vec(sentences=data, vector_size=100, window=5, min_count=1, workers=4)

print("Exemple de vocabulaire :", list(model_embedding.wv.key_to_index.keys())[:10])

[nltk_data] Downloading package punkt to /home/dimitri/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Exemple de vocabulaire : ['<pad>', 'should', 'we', 'the', 'to', '.', 'and', 'of', 'a', 'be']


In [ ]:
import numpy as np

corpus = [
    "Deep learning is powerful",
    "Neural networks are used for computer vision",
    "Word2Vec learns word representations"
]

def sentence_to_vec(sentence, model):
    
    """Convertit une phrase en un vecteur en moyennant les vecteurs Word2Vec des mots"""

    vectors = [model.wv[word] for word in sentence if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)


In [22]:
X = np.array([sentence_to_vec(sent, model_embedding) for sent in data])

In [39]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, np.array(note), test_size=0.2, random_state=42)

In [29]:
import tensorflow as tf


model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [37]:
type(X_train)

numpy.ndarray

In [40]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

Epoch 1/10
  1/525 [..............................] - ETA: 2s - loss: 0.5555 - accuracy: 0.1875

525/525 [==============================] - 1s 2ms/step - loss: 0.4976 - accuracy: 0.1955 - val_loss: 0.4972 - val_accuracy: 0.1979
Epoch 2/10
525/525 [==============================] - 1s 1ms/step - loss: 0.4972 - accuracy: 0.1954 - val_loss: 0.4967 - val_accuracy: 0.1979
Epoch 3/10
525/525 [==============================] - 1s 2ms/step - loss: 0.4962 - accuracy: 0.1955 - val_loss: 0.4964 - val_accuracy: 0.1979
Epoch 4/10
525/525 [==============================] - 1s 1ms/step - loss: 0.4955 - accuracy: 0.1955 - val_loss: 0.4957 - val_accuracy: 0.1979
Epoch 5/10
525/525 [==============================] - 1s 2ms/step - loss: 0.4951 - accuracy: 0.1954 - val_loss: 0.4962 - val_accuracy: 0.1979
Epoch 6/10
525/525 [==============================] - 1s 2ms/step - loss: 0.4943 - accuracy: 0.1954 - val_loss: 0.4970 - val_accuracy: 0.1979
Epoch 7/10
525/525 [==============================] - 1s 2ms/step - loss: 0.4936 - accuracy: 0.1955 - val_loss: 0.4958 - val_accuracy: 0.1979
Epoch 8/10
525/52